# __Fetch aoe2recs.com match details parallely__

### __Load libraries__

In [1]:
from pymongo import MongoClient

from tqdm import tqdm

from bs4 import BeautifulSoup
import json
import pandas as pd
import numpy as np
import os
import random

from itertools import cycle
import requests
import mechanize
import http.cookiejar
import random
import warnings

import multiprocessing
from joblib import Parallel, delayed

cores =  20 # API rate limiter forces to use 20
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

C:\Users\Parikshit_verma\Anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.25.11) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


### __Source functions__

In [2]:
%run aoe2rec_api_modules.ipynb

### __Fetch proxies & user agents__

In [43]:
proxy_list = fetch_proxies(https = False)
uas_list = fetch_user_agents()

### __Set up mongo connection__

In [ ]:
db_conn = setup_mongo_conn()

### __Delete invalid matches__

In [ ]:
delete_invalid_match_mongo(db_conn)

### __Fetch match details__

In [ ]:
fetch_multiple_matches_iterate_aoe2recs(db_conn, collect = 150, batch_size = 20, clean_empty = True, proxy_list = None, uas_list = None)